In [1]:
# Import needed Python modules 
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

import numpy as np

import konrad

In [2]:
#
# Used in examples below
#
atmosphere = konrad.atmosphere.Atmosphere(konrad.utils.get_quadratic_pgrid(1000e2, 10, 128))
atmosphere["CO2"][:] = 280E-6
initial_conds = atmosphere.copy()

### Feedbacks under fixed Q

In [3]:
Ts = 294

base = konrad.RCE(atmosphere = initial_conds.copy(), 
                 surface=konrad.surface.FixedTemperature(temperature=Ts))
base.run()
Neq = base.radiation["toa"][-1]

pert = konrad.RCE(atmosphere = base.atmosphere.copy(), 
                 surface=konrad.surface.FixedTemperature(temperature=Ts+1))
pert.run()

In [4]:
# Make a mask for tropospheric values
trop = base.atmosphere["plev"] > 230.*100.

In [5]:
# Planck feedback - uniform tropspheric warming, fixed Q
rad = copy.deepcopy(base) 
rad.atmosphere["T"][0,:][trop] += 1
rad.radiation.update_heatingrates(rad.atmosphere, rad.surface)
lambda_planck = rad.radiation["toa"][-1] - base.radiation["toa"][-1]
print("Const. Q, Planck feedback:", np.round(lambda_planck, 2), " W/m2-K")

Const. Q, Planck feedback: -2.83  W/m2-K


In [6]:
# Feedback from vertically-varying tropospheric change in T, fixed Q
rad = copy.deepcopy(base) 
rad.atmosphere["T"][0,:][trop] = pert.atmosphere["T"][0,:][trop]
rad.radiation.update_heatingrates(rad.atmosphere, rad.surface)
lambda_T = rad.radiation["toa"][-1] - base.radiation["toa"][-1]
print("Const. Q, total T feedback:", np.round(lambda_T, 2), " W/m2-K")
print("Const. Q, LR feedback:", np.round(lambda_T - lambda_planck, 2), " W/m2-K")

Const. Q, total T feedback: -4.72  W/m2-K
Const. Q, LR feedback: -1.89  W/m2-K


In [7]:
# Water vapor feedback - fixed Q
rad = copy.deepcopy(base) 
rad.atmosphere["H2O"][0,:][trop] = pert.atmosphere["H2O"][0,:][trop]
rad.radiation.update_heatingrates(rad.atmosphere, rad.surface)
print("Const. Q, water vapor feedback:", np.round(rad.radiation["toa"][-1] - base.radiation["toa"][-1], 2), " W/m2-K")

Const. Q, water vapor feedback: 4.26  W/m2-K


### Feedbacks with constant RH

In [9]:
# Planck feedback 
rad = copy.deepcopy(base)
rad.atmosphere["T"][0,:][trop] += 1
# Impose fixed RH
rad.atmosphere["H2O"][0,:][trop] = \
    konrad.physics.relative_humidity2vmr(0.8, rad.atmosphere["plev"][trop], rad.atmosphere["T"][0,trop])
rad.radiation.update_heatingrates(rad.atmosphere, rad.surface)
lambda_planck = rad.radiation["toa"][-1] - base.radiation["toa"][-1]
print("Const. RH, Planck feedback:", np.round(lambda_planck, 2), " W/m2-K")

# Feedback from total tropospheric change in T, fixed RH
rad = copy.deepcopy(base)
# Impose fixed RH - guaranteed by konrad 
rad.atmosphere["H2O"][0,:][trop] = pert.atmosphere["H2O"][0,:][trop]
rad.atmosphere["T"]  [0,:][trop] = pert.atmosphere["T"]  [0,:][trop]
rad.radiation.update_heatingrates(rad.atmosphere, rad.surface)
lambda_T = rad.radiation["toa"][-1] - base.radiation["toa"][-1]
print("Const. RH, total T feedback:", np.round(lambda_T, 2), " W/m2-K")

print("Const. RH, LR feedback:", np.round(lambda_T - lambda_planck, 2), " W/m2-K")

print("Const. RH: water vapor feedback is 0 by construction") 

Const. RH, Planck feedback: -0.5  W/m2-K
Const. RH, total T feedback: -0.62  W/m2-K
Const. RH, LR feedback: -0.13  W/m2-K
Const. RH: water vapor feedback is 0 by construction
